In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import os
os.chdir(r"D:\coding\Jupyter_notebooks\ML")
from credit_risk.read_data import read_data_train
from credit_risk.transform_pipeline import data_train_pipeline

In [7]:
data_original = read_data_train()

In [14]:
data_train = data_train_pipeline.fit_transform(data_original.copy(deep=True))
data_train = data_train.astype("float32")

In [25]:
data_train.dtypes.value_counts()

float32    121
dtype: int64

In [15]:
data_train["AMT_INCOME_TOTAL"].describe()

count    177039.000000
mean          2.573936
std           1.731862
min           0.000000
25%           1.000000
50%           2.000000
75%           4.000000
max           6.000000
Name: AMT_INCOME_TOTAL, dtype: float64

In [20]:
y_columns = ["TARGET"]

def get_data_xy(df):
    return df[df.columns.difference(y_columns)], df[y_columns]

In [34]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    score = 2 * (precision * recall) / (precision + recall)
    return score

credit_default_model = keras.Sequential([
    keras.Input(shape=(len(data_train.columns) - len(y_columns),), name="input"),
    layers.Dense(50, activation="linear", name="dense_1"),
    layers.Dense(10, activation="relu", name="dense_2"),
    layers.Dense(1, activation="sigmoid", name="output")
], name="credit_default_model")

credit_default_model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.Adam(0.01),
              metrics=[f1_score])

In [35]:
credit_default_model.summary()

Model: "credit_default_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                6050      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
_________________________________________________________________
output (Dense)               (None, 1)                 11        
Total params: 6,571
Trainable params: 6,571
Non-trainable params: 0
_________________________________________________________________


In [39]:
from sklearn.model_selection import train_test_split

def train_model(model, x, y, epochs = 10):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, stratify=y)

    history = model.fit(x_train, y_train, 500, epochs, validation_split=0.2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    print("f1 score:", f1_score(np.asarray(y_test), model.predict(x_test)).numpy())

In [40]:
x_data, y_data = get_data_xy(data_train)

In [41]:
train_model(credit_default_model, x_data, y_data)

Epoch 1/10


TypeError: in user code:

    c:\users\anton\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-19-4f038cb8055d>:13 f1_score  *
        if c3 == 0:
    c:\users\anton\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\autograph\operators\control_flow.py:1163 if_stmt
        _tf_if_stmt(cond, body, orelse, get_state, set_state, symbol_names, nouts)
    c:\users\anton\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\autograph\operators\control_flow.py:1209 _tf_if_stmt
        final_cond_vars = control_flow_ops.cond(
    c:\users\anton\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper
        return target(*args, **kwargs)
    c:\users\anton\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\util\deprecation.py:538 new_func
      

In [ ]:
count_class0, count_class1 = data_train[y_columns].value_counts()

In [ ]:
count_class1

In [ ]:
# test with undersampling

